In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH
import pandas as pd
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

In [ ]:
train_tr.shape , train_target.shape , test_tr.shape , submit.shape
# from IPython.display import display, HTML
# br_html = HTML("<br><br>")
# display(HTML("<h1>학습용 구매기록 데이터</h1>"), train_tr.head(), br_html)
# display(HTML("<h1>학습용 정답 데이터</h1>"), train_target.head(), br_html)
# display(HTML("<h1>테스트용 구매기록 데이터</h1>"), test_tr.head(), br_html)
# display(HTML("<h1>제출 양식 데이터</h1>"), submit.head(), br_html)
# display(HTML("<h1>학습용 구매기록 데이터</h1>"))
# display(train_tr.info(), br_html)

# display(HTML("<h1>학습용 정답 데이터</h1>"))
# display(train_target.info(), br_html)

# display(HTML("<h1>테스트용 구매기록 데이터</h1>"))
# display(test_tr.info(), br_html)

# display(HTML("<h1>제출 양식 데이터</h1>"))
# display(submit.info(), br_html)
train_target["target"].mean()
train_tr["ID"].nunique() , train_target.shape
train_tr.sort_values("ID").head(50)
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])
time_delta = train_tr["구매일시"].max() - train_tr["구매일시"].min()
time_delta
train_ft = train_target[["ID"]]
test_ft = submit[["ID"]]

train_ft.shape, test_ft.shape
agg_list = [
        # 컬럼명, 집계 방식
        ('내점일수','nunique'),
        ('구매주기', lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique()) ),
        ('주말방문비율', lambda x: np.mean(x.dt.weekday>4)),
        ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
        ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
        ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
        ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
        ('주구매요일', lambda x: x.dt.weekday.mode()[0]),
        ('일별평균구매건수', lambda x:  x.count() / x.dt.date.nunique() ),
        ('거래개월수', lambda x: x.dt.date.astype(str).str[:-3].nunique() ),
    ]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

agg_list = [
          ("방문지점수","nunique"),
          ('주구매지점', lambda x: x.mode()[0]),
    ]

tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()
tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()
agg_list = [
             ('브랜드코드_nunique', 'nunique'),
             ]

tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()
tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()
agg_list = [
            ('중분류_nunique', 'nunique'),
            ('주구매_중분류', lambda x: x.mode()[0]),
            ]
tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()
tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()
agg_list = [
            ('대분류_nunique', 'nunique'),
            ('주구매_대분류', lambda x: x.mode()[0]),
            ]

tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()
tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()
agg_list = [
        ('총구매액','sum'),
        ('구매건수', 'count'),
        ('평균구매액', "mean"),
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ),
        ('구매금액표준편차',lambda x: x[x>0].std() ),
    ]

tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()
tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

train_ft = train_ft.to_csv(f'{DATA_PATH}train_ft.csv', index = False)
test_ft = test_ft.to_csv(f'{DATA_PATH}test_ft.csv', index = False)

In [ ]:
# 구매일시 지점코드 대분류 중분류 브랜드코드 구매 가격
# 구매 가격 > regular, vip 등급나누기
# 지점코드 > 기피지점

In [ ]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_ft = pd.read_csv(f'{DATA_PATH}train_ft.csv')
test_ft = pd.read_csv(f'{DATA_PATH}test_ft.csv')

In [ ]:
train_tr.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000


In [ ]:
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,대분류_nunique,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,9,남성정장스포츠,4201200,20,210060.000000,1236000,20000,-2517000,3,403660.245259
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,12,영플라자,5043025,42,120072.023810,698000,5600,0,0,126592.626144
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,14,명품잡화,22135386,114,194170.052632,2770000,4400,-1072500,7,406609.265932
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,16,케주얼_구두_아동,35594762,211,168695.554502,1492000,10000,-2867800,14,236976.860907
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,8,명품잡화,3988866,62,64336.548387,1204000,7640,-6954400,13,308503.944915


In [ ]:
train_tr.shape, train_ft.shape, test_ft.shape

((523105, 7), (14940, 26), (12225, 26))

In [ ]:
len(train_ft) * 0.1

1494.0

In [ ]:
total_sum = train_ft['총구매액'].sort_values(ascending = False).reset_index()
total_sum

,index,총구매액
0,11204,99190685
1,12338,85098557
2,3924,82957000
3,11804,82094852
4,12262,72067270
...,...,...
14935,9347,-131000
14936,9102,-404945
14937,13327,-727800
14938,7681,-1453576


In [ ]:
rich = int(len(total_sum) * 0.1)
rich

1494

In [ ]:
vip_class = total_sum.loc[ : rich, : ]
vip_class

,index,총구매액
0,11204,99190685
1,12338,85098557
2,3924,82957000
3,11804,82094852
4,12262,72067270
...,...,...
1490,3010,8230625
1491,1319,8221660
1492,11753,8221040
1493,795,8210163


In [ ]:
vip_class['총구매액'].min()

8209000

In [ ]:
vip_cut = train_ft['총구매액'] >= vip_class['총구매액'].min()
vip_cut

,총구매액
0,False
1,False
2,True
3,True
4,False
...,...
14935,False
14936,False
14937,False
14938,False


In [ ]:
regular_cut = train_ft['총구매액'] < vip_class['총구매액'].min()
regular_cut

,총구매액
0,True
1,True
2,False
3,False
4,True
...,...
14935,True
14936,True
14937,True
14938,True


In [ ]:
vip_customer = train_ft.loc[vip_cut, : ]

# vip_customer = vip_customer[['ID']]
# type(vip_customer)
# vip_customer = vip_customer.rename(columns = {'ID': 'vip'})
vip_customer

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,대분류_nunique,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,14,명품잡화,22135386,114,194170.052632,2770000,4400,-1072500,7,406609.265932
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,16,케주얼_구두_아동,35594762,211,168695.554502,1492000,10000,-2867800,14,236976.860907
5,train_5,87,4,0.287234,0.202128,0.287234,0.244681,0.265957,6,1.362319,...,9,잡화파트,10047455,94,106887.819149,1200000,8900,-2124000,7,191785.829583
10,train_10,113,8,0.022388,0.156716,0.223881,0.358209,0.261194,4,3.045455,...,10,공산품파트,11234870,134,83842.313433,907000,2500,-192750,3,142331.010505
35,train_35,78,5,0.354430,0.189873,0.240506,0.227848,0.341772,0,1.295082,...,9,공산품,9159550,79,115943.670886,2428000,7600,0,0,308722.548733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14892,train_14892,68,10,0.187500,0.212500,0.525000,0.100000,0.162500,4,2.580645,...,15,케주얼_구두_아동,12410223,80,155127.787500,3000000,5300,-1319800,7,402768.194488
14895,train_14895,43,15,0.260870,0.282609,0.347826,0.282609,0.086957,2,2.300000,...,9,명품잡화,8360200,46,181743.478261,1309000,13000,-1310000,3,281176.870458
14907,train_14907,32,8,0.031250,0.531250,0.000000,0.031250,0.437500,3,1.684211,...,6,잡화파트,13629521,32,425922.531250,2962800,12500,0,0,682330.540324
14917,train_14917,124,8,0.135338,0.187970,0.375940,0.255639,0.180451,2,3.093023,...,9,잡화파트,35884744,133,269810.105263,2999000,8000,-4983000,5,539628.646687


In [ ]:
regular_customer = train_ft.loc[regular_cut, : ]
regular_customer

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,대분류_nunique,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,9,남성정장스포츠,4201200,20,210060.000000,1236000,20000,-2517000,3,403660.245259
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,12,영플라자,5043025,42,120072.023810,698000,5600,0,0,126592.626144
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,8,명품잡화,3988866,62,64336.548387,1204000,7640,-6954400,13,308503.944915
6,train_6,11,52,0.363636,0.454545,0.090909,0.363636,0.090909,4,1.833333,...,6,여성정장,2785379,11,253216.272727,953200,19098,0,0,349657.075502
7,train_7,46,18,0.098039,0.352941,0.176471,0.137255,0.333333,4,2.833333,...,12,가정용품,2533925,51,49684.803922,1020000,10400,-2098000,3,161093.259131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_14935,16,24,0.210526,0.000000,0.578947,0.052632,0.368421,2,1.900000,...,7,잡화,1195120,19,62901.052632,217000,12000,-122000,1,68478.218778
14936,train_14936,8,29,0.125000,0.125000,0.625000,0.250000,0.000000,0,1.000000,...,5,골프_유니캐쥬얼,2220000,8,277500.000000,1894000,52000,-294000,2,725920.932334
14937,train_14937,22,26,0.083333,0.208333,0.458333,0.208333,0.125000,1,2.000000,...,6,공산품,897898,24,37412.416667,116100,6000,0,0,32517.709765
14938,train_14938,31,28,0.151515,0.393939,0.393939,0.212121,0.000000,3,2.750000,...,3,영플라자,4136600,33,125351.515152,424800,1000,0,0,103530.506136


In [ ]:
type(regular_customer)

pandas.core.frame.DataFrame

In [ ]:
train_ft['vip_class'] = vip_customer['ID']
train_ft['vip_class'] = train_ft['vip_class'].astype(object)
train_ft['vip_class'] = train_ft['vip_class'].fillna('not')


In [ ]:
train_ft

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,vip_class
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,남성정장스포츠,4201200,20,210060.000000,1236000,20000,-2517000,3,403660.245259,not
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,영플라자,5043025,42,120072.023810,698000,5600,0,0,126592.626144,not
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,명품잡화,22135386,114,194170.052632,2770000,4400,-1072500,7,406609.265932,train_2
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,케주얼_구두_아동,35594762,211,168695.554502,1492000,10000,-2867800,14,236976.860907,train_3
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,명품잡화,3988866,62,64336.548387,1204000,7640,-6954400,13,308503.944915,not
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_14935,16,24,0.210526,0.000000,0.578947,0.052632,0.368421,2,1.900000,...,잡화,1195120,19,62901.052632,217000,12000,-122000,1,68478.218778,not
14936,train_14936,8,29,0.125000,0.125000,0.625000,0.250000,0.000000,0,1.000000,...,골프_유니캐쥬얼,2220000,8,277500.000000,1894000,52000,-294000,2,725920.932334,not
14937,train_14937,22,26,0.083333,0.208333,0.458333,0.208333,0.125000,1,2.000000,...,공산품,897898,24,37412.416667,116100,6000,0,0,32517.709765,not
14938,train_14938,31,28,0.151515,0.393939,0.393939,0.212121,0.000000,3,2.750000,...,영플라자,4136600,33,125351.515152,424800,1000,0,0,103530.506136,not


In [ ]:
test_ft['vip_class'] = vip_customer['ID']
test_ft['vip_class'] = test_ft['vip_class'].astype(object)
test_ft['vip_class'] = test_ft['vip_class'].fillna('not')

In [ ]:
train_ft

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,vip_class
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,남성정장스포츠,4201200,20,210060.000000,1236000,20000,-2517000,3,403660.245259,not
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,영플라자,5043025,42,120072.023810,698000,5600,0,0,126592.626144,not
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,명품잡화,22135386,114,194170.052632,2770000,4400,-1072500,7,406609.265932,train_2
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,케주얼_구두_아동,35594762,211,168695.554502,1492000,10000,-2867800,14,236976.860907,train_3
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,명품잡화,3988866,62,64336.548387,1204000,7640,-6954400,13,308503.944915,not
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_14935,16,24,0.210526,0.000000,0.578947,0.052632,0.368421,2,1.900000,...,잡화,1195120,19,62901.052632,217000,12000,-122000,1,68478.218778,not
14936,train_14936,8,29,0.125000,0.125000,0.625000,0.250000,0.000000,0,1.000000,...,골프_유니캐쥬얼,2220000,8,277500.000000,1894000,52000,-294000,2,725920.932334,not
14937,train_14937,22,26,0.083333,0.208333,0.458333,0.208333,0.125000,1,2.000000,...,공산품,897898,24,37412.416667,116100,6000,0,0,32517.709765,not
14938,train_14938,31,28,0.151515,0.393939,0.393939,0.212121,0.000000,3,2.750000,...,영플라자,4136600,33,125351.515152,424800,1000,0,0,103530.506136,not


In [ ]:
train_ft['regular_class'] = regular_customer['ID']
train_ft['regular_class'] = train_ft['regular_class'].fillna('not')

In [ ]:
train_ft

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,vip_class,regular_class
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,4201200,20,210060.000000,1236000,20000,-2517000,3,403660.245259,not,train_0
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,5043025,42,120072.023810,698000,5600,0,0,126592.626144,not,train_1
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,22135386,114,194170.052632,2770000,4400,-1072500,7,406609.265932,train_2,not
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,35594762,211,168695.554502,1492000,10000,-2867800,14,236976.860907,train_3,not
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,3988866,62,64336.548387,1204000,7640,-6954400,13,308503.944915,not,train_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_14935,16,24,0.210526,0.000000,0.578947,0.052632,0.368421,2,1.900000,...,1195120,19,62901.052632,217000,12000,-122000,1,68478.218778,not,train_14935
14936,train_14936,8,29,0.125000,0.125000,0.625000,0.250000,0.000000,0,1.000000,...,2220000,8,277500.000000,1894000,52000,-294000,2,725920.932334,not,train_14936
14937,train_14937,22,26,0.083333,0.208333,0.458333,0.208333,0.125000,1,2.000000,...,897898,24,37412.416667,116100,6000,0,0,32517.709765,not,train_14937
14938,train_14938,31,28,0.151515,0.393939,0.393939,0.212121,0.000000,3,2.750000,...,4136600,33,125351.515152,424800,1000,0,0,103530.506136,not,train_14938


In [ ]:
test_ft['regular_class'] = regular_customer['ID']
test_ft['regular_class'] = test_ft['regular_class'].fillna('not')

In [ ]:
test_ft

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,vip_class,regular_class
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,429579,7,61368.428571,110000,26643,0,0,32293.148313,not,train_0
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,290600,4,72650.000000,118000,37600,0,0,33658.431336,not,train_1
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,607000,5,121400.000000,326000,38000,0,0,117233.954126,train_2,not
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,2305740,32,72054.375000,403000,7040,-1092000,4,122556.774972,train_3,not
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,1568690,21,74699.523810,376000,16200,-372200,3,94997.232393,not,train_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_12220,12,47,0.000000,0.166667,0.166667,0.250000,0.416667,2,2.000000,...,1600500,12,133375.000000,328000,11500,0,0,89241.532575,not,train_12220
12221,test_12221,19,24,0.238095,0.380952,0.000000,0.238095,0.380952,0,1.500000,...,792100,21,37719.047619,230400,21000,-404600,3,48012.721966,not,train_12221
12222,test_12222,38,12,0.315789,0.157895,0.263158,0.289474,0.289474,2,1.357143,...,14542358,38,382693.631579,1256000,32538,0,0,333404.594689,not,train_12222
12223,test_12223,221,3,0.202335,0.171206,0.311284,0.287938,0.229572,1,2.234783,...,15998505,257,62250.992218,856000,3000,-6291400,27,117819.526276,not,train_12223


In [ ]:
train_ft.isnull().sum()
test_ft.isnull().sum()
train_ft = train_ft.fillna(0)
test_ft = test_ft.fillna(0)

In [ ]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [ ]:
train_ft = train_ft.drop(columns="ID")
test_ft = test_ft.drop(columns="ID")

In [ ]:
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,vip_class,regular_class
0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,...,4201200,20,210060.000000,1236000,20000,-2517000,3,403660.245259,not,train_0
1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,...,5043025,42,120072.023810,698000,5600,0,0,126592.626144,not,train_1
2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,...,22135386,114,194170.052632,2770000,4400,-1072500,7,406609.265932,train_2,not
3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,...,35594762,211,168695.554502,1492000,10000,-2867800,14,236976.860907,train_3,not
4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,...,3988866,62,64336.548387,1204000,7640,-6954400,13,308503.944915,not,train_4


In [ ]:
test_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,vip_class,regular_class
0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,...,429579,7,61368.428571,110000,26643,0,0,32293.148313,not,train_0
1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,...,290600,4,72650.000000,118000,37600,0,0,33658.431336,not,train_1
2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,...,607000,5,121400.000000,326000,38000,0,0,117233.954126,train_2,not
3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,...,2305740,32,72054.375000,403000,7040,-1092000,4,122556.774972,train_3,not
4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,...,1568690,21,74699.523810,376000,16200,-372200,3,94997.232393,not,train_4


In [ ]:
cols = train_ft.select_dtypes("object").columns.tolist()
cols

['주구매지점', '주구매_중분류', '주구매_대분류', 'vip_class', 'regular_class']

In [ ]:
train_ft['주구매_중분류']

,주구매_중분류
0,골프웨어
1,영캐주얼
2,아웃도어
3,수입종합화장품
4,로얄부틱2F
...,...
14935,국산화장품
14936,골프(국내)
14937,건강식품
14938,영캐주얼


In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown="ignore")

In [ ]:
enc.fit_transform(train_ft[cols]).toarray()

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
tmp = pd.DataFrame(
    enc.fit_transform(train_ft[cols]).toarray(),
    columns =  enc.get_feature_names_out()
)

train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,regular_class_train_999,regular_class_train_9990,regular_class_train_9991,regular_class_train_9992,regular_class_train_9993,regular_class_train_9995,regular_class_train_9996,regular_class_train_9997,regular_class_train_9998,regular_class_train_9999
0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)

In [ ]:
test_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,regular_class_train_999,regular_class_train_9990,regular_class_train_9991,regular_class_train_9992,regular_class_train_9993,regular_class_train_9995,regular_class_train_9996,regular_class_train_9997,regular_class_train_9998,regular_class_train_9999
0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,regular_class_train_999,regular_class_train_9990,regular_class_train_9991,regular_class_train_9992,regular_class_train_9993,regular_class_train_9995,regular_class_train_9996,regular_class_train_9997,regular_class_train_9998,regular_class_train_9999
0,0.027778,0.130682,0.250000,0.050000,0.250000,0.400000,0.300000,0.500000,0.090909,0.545455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.074074,0.090909,0.023810,0.357143,0.166667,0.357143,0.119048,0.500000,0.111111,0.909091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.187037,0.034091,0.210526,0.464912,0.140351,0.175439,0.219298,0.000000,0.115079,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.351852,0.017045,0.189573,0.379147,0.180095,0.236967,0.203791,0.500000,0.143720,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.100000,0.062500,0.258065,0.112903,0.612903,0.209677,0.064516,0.666667,0.144033,0.818182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_ft[test_ft.columns] = scaler.fit_transform(test_ft)
test_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,regular_class_train_999,regular_class_train_9990,regular_class_train_9991,regular_class_train_9992,regular_class_train_9993,regular_class_train_9995,regular_class_train_9996,regular_class_train_9997,regular_class_train_9998,regular_class_train_9999
0,0.013921,0.188235,0.571429,0.285714,0.285714,0.428571,0.000000,0.166667,0.050000,0.272727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.006961,0.317647,0.250000,0.000000,0.000000,0.750000,0.250000,0.000000,0.041667,0.181818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.006961,0.047059,0.000000,0.000000,0.600000,0.400000,0.000000,0.000000,0.187500,0.090909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.062645,0.094118,0.031250,0.062500,0.468750,0.343750,0.125000,0.500000,0.125000,0.545455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.041763,0.170588,0.476190,0.380952,0.285714,0.238095,0.095238,1.000000,0.166667,0.272727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_ft, target, test_size=0.2, random_state=42)

x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((11952, 15242), (2988, 15242), (11952,), (2988,))

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model.fit(x_train,y_train)
pred = model.predict(x_valid)
pred[:5]

array([0., 1., 1., 1., 0.])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid,pred)

In [ ]:
model = LogisticRegression(random_state=42)
model.fit(train_ft,target)
pred = model.predict(test_ft)
pred

In [ ]:
submit["target"] = pred
submit
submit.to_csv("고객등급.csv",index=False)

NameError: name 'pred' is not defined